In [1]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
import catboost as cb

In [2]:
train = pd.read_csv("Train_Data.csv")
test = pd.read_csv("Test_Data.csv")

In [3]:
train.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train['adgroup'] = le.fit_transform(train['adgroup'])
test['adgroup'] = le.transform(test['adgroup'])

train = train.set_index(train['date'])
test = test.set_index(test['date'])

train.drop(['date', 'campaign'],axis=1,inplace=True)
test.drop(['date','campaign'],axis=1,inplace=True)


In [5]:
for i in range(len(train)):
    train['ad'][i] = int(train['ad'][i][3:])
for i in range(len(test)):
    test['ad'][i]=int(test['ad'][i][3:])
train['ad']=pd.to_numeric(train['ad'])
test['ad']=pd.to_numeric(test['ad'])
# train.drop(['ad'],axis=1,inplace=True)

<ipython-input-5-a1ae5a052bdc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['ad'][i] = int(train['ad'][i][3:])
<ipython-input-5-a1ae5a052bdc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ad'][i]=int(test['ad'][i][3:])


In [8]:
train['CTR']=train['clicks']/train['impressions']
train['CPC']=train['cost']/train['clicks']
train['CPA']=train['cost']/train['conversions']

test['CTR']=test['clicks']/test['impressions']
test['CPC']=test['cost']/test['clicks']
test['CPA']=test['cost']/test['conversions']

train['CPC'].fillna(0,inplace=True)
train['CPA'].fillna(0,inplace=True)

test['CPC'].fillna(0,inplace=True)
test['CPA'].fillna(0,inplace=True)

train.replace([np.inf], 65,inplace=True)
test.replace([np.inf], 65,inplace=True)

In [9]:
X_train = train.drop(['revenue'], axis=1)
y_train = train.revenue
X_test = test
train_dataset = cb.Pool(X_train, y_train)

In [10]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRFRegressor, XGBRegressor

In [11]:
model = cb.CatBoostRegressor(loss_function='RMSE')

grid = {'iterations': [1000, 1500],
        'learning_rate': [0.01],
        'depth': [2, 4],
        'l2_leaf_reg': [0.2, 0.5]}
model.grid_search(grid, train_dataset)

0:	learn: 1136.4534278	test: 977.4735087	best: 977.4735087 (0)	total: 140ms	remaining: 2m 19s
1:	learn: 1127.2419978	test: 970.3862508	best: 970.3862508 (1)	total: 142ms	remaining: 1m 10s
2:	learn: 1118.0138484	test: 959.8216614	best: 959.8216614 (2)	total: 144ms	remaining: 47.9s
3:	learn: 1109.2551096	test: 951.4556302	best: 951.4556302 (3)	total: 146ms	remaining: 36.4s
4:	learn: 1100.3918986	test: 942.7506126	best: 942.7506126 (4)	total: 148ms	remaining: 29.5s
5:	learn: 1091.8752705	test: 933.4745754	best: 933.4745754 (5)	total: 150ms	remaining: 24.8s
6:	learn: 1083.4241849	test: 925.1536266	best: 925.1536266 (6)	total: 151ms	remaining: 21.5s
7:	learn: 1075.1695808	test: 917.0024579	best: 917.0024579 (7)	total: 153ms	remaining: 19s
8:	learn: 1066.6019472	test: 906.8679925	best: 906.8679925 (8)	total: 154ms	remaining: 17s
9:	learn: 1058.4276838	test: 897.8904878	best: 897.8904878 (9)	total: 155ms	remaining: 15.4s
10:	learn: 1049.8800931	test: 888.2127522	best: 888.2127522 (10)	total: 

204:	learn: 330.2116438	test: 283.2678714	best: 283.2678714 (204)	total: 313ms	remaining: 1.21s
205:	learn: 328.8571320	test: 282.5052569	best: 282.5052569 (205)	total: 314ms	remaining: 1.21s
206:	learn: 327.6199344	test: 282.0545090	best: 282.0545090 (206)	total: 315ms	remaining: 1.21s
207:	learn: 326.4666869	test: 281.6013047	best: 281.6013047 (207)	total: 315ms	remaining: 1.2s
208:	learn: 325.3000115	test: 280.9594792	best: 280.9594792 (208)	total: 316ms	remaining: 1.2s
209:	learn: 324.2872518	test: 280.4968936	best: 280.4968936 (209)	total: 317ms	remaining: 1.19s
210:	learn: 323.0044194	test: 279.8845864	best: 279.8845864 (210)	total: 317ms	remaining: 1.19s
211:	learn: 321.7859489	test: 279.2906796	best: 279.2906796 (211)	total: 318ms	remaining: 1.18s
212:	learn: 320.6450577	test: 278.6979056	best: 278.6979056 (212)	total: 319ms	remaining: 1.18s
213:	learn: 319.3734588	test: 278.0542957	best: 278.0542957 (213)	total: 319ms	remaining: 1.17s
214:	learn: 318.0868927	test: 277.3329058	

467:	learn: 195.0520970	test: 228.4444599	best: 228.4444599 (467)	total: 488ms	remaining: 554ms
468:	learn: 194.8383887	test: 228.1830152	best: 228.1830152 (468)	total: 489ms	remaining: 553ms
469:	learn: 194.7325512	test: 228.1662054	best: 228.1662054 (469)	total: 489ms	remaining: 552ms
470:	learn: 194.6450660	test: 228.1502502	best: 228.1502502 (470)	total: 490ms	remaining: 550ms
471:	learn: 194.3329383	test: 227.7844544	best: 227.7844544 (471)	total: 491ms	remaining: 549ms
472:	learn: 194.1721976	test: 227.7369016	best: 227.7369016 (472)	total: 491ms	remaining: 548ms
473:	learn: 194.0844808	test: 227.7171255	best: 227.7171255 (473)	total: 492ms	remaining: 546ms
474:	learn: 193.9856583	test: 227.6749701	best: 227.6749701 (474)	total: 493ms	remaining: 545ms
475:	learn: 193.5702745	test: 227.5132129	best: 227.5132129 (475)	total: 493ms	remaining: 543ms
476:	learn: 193.3196426	test: 227.3099502	best: 227.3099502 (476)	total: 494ms	remaining: 542ms
477:	learn: 193.1526394	test: 227.110845

735:	learn: 166.9821993	test: 209.7455921	best: 209.7455921 (735)	total: 663ms	remaining: 238ms
736:	learn: 166.8349088	test: 209.5246683	best: 209.5246683 (736)	total: 663ms	remaining: 237ms
737:	learn: 166.7812670	test: 209.5384068	best: 209.5246683 (736)	total: 664ms	remaining: 236ms
738:	learn: 166.6805766	test: 209.4936475	best: 209.4936475 (738)	total: 665ms	remaining: 235ms
739:	learn: 166.6406065	test: 209.3755453	best: 209.3755453 (739)	total: 665ms	remaining: 234ms
740:	learn: 166.6032544	test: 209.2184707	best: 209.2184707 (740)	total: 666ms	remaining: 233ms
741:	learn: 166.5869826	test: 209.1503550	best: 209.1503550 (741)	total: 667ms	remaining: 232ms
742:	learn: 166.5556911	test: 209.1101624	best: 209.1101624 (742)	total: 667ms	remaining: 231ms
743:	learn: 166.5151734	test: 209.0219629	best: 209.0219629 (743)	total: 668ms	remaining: 230ms
744:	learn: 166.4792088	test: 209.0155942	best: 209.0155942 (744)	total: 669ms	remaining: 229ms
745:	learn: 166.4184312	test: 208.937703

1:	learn: 1127.6103909	test: 970.7011309	best: 970.7011309 (1)	total: 1.65ms	remaining: 822ms
2:	learn: 1118.5316578	test: 960.3223282	best: 960.3223282 (2)	total: 2.3ms	remaining: 764ms
3:	learn: 1109.8711947	test: 952.0493747	best: 952.0493747 (3)	total: 3.21ms	remaining: 801ms
4:	learn: 1101.1129144	test: 943.4471478	best: 943.4471478 (4)	total: 3.88ms	remaining: 773ms
5:	learn: 1092.6965917	test: 934.2852946	best: 934.2852946 (5)	total: 4.54ms	remaining: 753ms
6:	learn: 1084.3428851	test: 926.0597278	best: 926.0597278 (6)	total: 5.19ms	remaining: 736ms
7:	learn: 1076.1828634	test: 918.0011962	best: 918.0011962 (7)	total: 5.82ms	remaining: 722ms
8:	learn: 1067.7431198	test: 908.0295171	best: 908.0295171 (8)	total: 6.44ms	remaining: 709ms
9:	learn: 1059.6595155	test: 899.1546485	best: 899.1546485 (9)	total: 7.06ms	remaining: 699ms
10:	learn: 1051.2420821	test: 889.6347364	best: 889.6347364 (10)	total: 7.66ms	remaining: 689ms
11:	learn: 1043.3380284	test: 880.5380508	best: 880.5380508

...

1287:	learn: 118.8053763	test: 186.0033290	best: 184.0539352 (927)	total: 1.21s	remaining: 199ms
1288:	learn: 118.7818601	test: 186.0643918	best: 184.0539352 (927)	total: 1.21s	remaining: 198ms
1289:	learn: 118.7633794	test: 186.0655374	best: 184.0539352 (927)	total: 1.21s	remaining: 197ms
1290:	learn: 118.7357770	test: 186.0280520	best: 184.0539352 (927)	total: 1.21s	remaining: 196ms
1291:	learn: 118.7070709	test: 186.0453700	best: 184.0539352 (927)	total: 1.21s	remaining: 195ms
1292:	learn: 118.6890375	test: 186.0433764	best: 184.0539352 (927)	total: 1.21s	remaining: 194ms
1293:	learn: 118.6718553	test: 186.0412861	best: 184.0539352 (927)	total: 1.21s	remaining: 193ms
1294:	learn: 118.6342776	test: 186.0421533	best: 184.0539352 (927)	total: 1.21s	remaining: 192ms
1295:	learn: 118.6059157	test: 186.0617170	best: 184.0539352 (927)	total: 1.21s	remaining: 191ms
1296:	learn: 118.5655027	test: 186.0899948	best: 184.0539352 (927)	total: 1.22s	remaining: 190ms
1297:	learn: 118.5337323	test:

1474:	learn: 114.5650651	test: 186.8542226	best: 184.0539352 (927)	total: 1.38s	remaining: 23.5ms
1475:	learn: 114.5427843	test: 186.8451921	best: 184.0539352 (927)	total: 1.39s	remaining: 22.5ms
1476:	learn: 114.5055697	test: 186.8360417	best: 184.0539352 (927)	total: 1.39s	remaining: 21.6ms
1477:	learn: 114.4903552	test: 186.8555594	best: 184.0539352 (927)	total: 1.39s	remaining: 20.7ms
1478:	learn: 114.4786037	test: 186.9064612	best: 184.0539352 (927)	total: 1.39s	remaining: 19.7ms
1479:	learn: 114.4535341	test: 186.9152184	best: 184.0539352 (927)	total: 1.39s	remaining: 18.8ms
1480:	learn: 114.4273885	test: 186.9131345	best: 184.0539352 (927)	total: 1.39s	remaining: 17.8ms
1481:	learn: 114.4092241	test: 186.8940358	best: 184.0539352 (927)	total: 1.39s	remaining: 16.9ms
1482:	learn: 114.3865020	test: 186.9003114	best: 184.0539352 (927)	total: 1.39s	remaining: 16ms
1483:	learn: 114.3749152	test: 186.9508410	best: 184.0539352 (927)	total: 1.39s	remaining: 15ms
1484:	learn: 114.3407316

206:	learn: 253.4648676	test: 316.4266994	best: 316.4266994 (206)
207:	learn: 252.2946186	test: 315.3759547	best: 315.3759547 (207)
208:	learn: 251.0958832	test: 314.3198312	best: 314.3198312 (208)
209:	learn: 249.9089814	test: 313.4384456	best: 313.4384456 (209)
210:	learn: 248.8532548	test: 312.6339861	best: 312.6339861 (210)
211:	learn: 247.6490329	test: 311.7261013	best: 311.7261013 (211)
212:	learn: 246.4562673	test: 310.7701382	best: 310.7701382 (212)
213:	learn: 245.3798882	test: 309.9679396	best: 309.9679396 (213)
214:	learn: 244.1984662	test: 308.9019951	best: 308.9019951 (214)
215:	learn: 243.1276572	test: 308.1196690	best: 308.1196690 (215)
216:	learn: 242.0079417	test: 307.2789608	best: 307.2789608 (216)
217:	learn: 240.9624026	test: 306.3515580	best: 306.3515580 (217)
218:	learn: 239.8575116	test: 305.5653626	best: 305.5653626 (218)
219:	learn: 238.8160464	test: 304.6686884	best: 304.6686884 (219)
220:	learn: 237.7244942	test: 303.7005577	best: 303.7005577 (220)
221:	learn

677:	learn: 136.6827567	test: 214.3789273	best: 214.3789273 (677)
678:	learn: 136.6284473	test: 214.3788036	best: 214.3788036 (678)
679:	learn: 136.5662692	test: 214.3689793	best: 214.3689793 (679)
680:	learn: 136.5255524	test: 214.3752895	best: 214.3689793 (679)
681:	learn: 136.4847704	test: 214.3454751	best: 214.3454751 (681)
682:	learn: 136.4242284	test: 214.2876611	best: 214.2876611 (682)
683:	learn: 136.3874778	test: 214.2755984	best: 214.2755984 (683)
684:	learn: 136.3387960	test: 214.2508749	best: 214.2508749 (684)
685:	learn: 136.2903893	test: 214.2215786	best: 214.2215786 (685)
686:	learn: 136.2413804	test: 214.2257365	best: 214.2215786 (685)
687:	learn: 136.1838361	test: 214.2135437	best: 214.2135437 (687)
688:	learn: 136.1301238	test: 214.1664728	best: 214.1664728 (688)
689:	learn: 136.0668988	test: 214.1442204	best: 214.1442204 (689)
690:	learn: 136.0121678	test: 214.1008542	best: 214.1008542 (690)
691:	learn: 135.9619875	test: 214.1050319	best: 214.1008542 (690)
692:	learn

{'params': {'depth': 4,
  'iterations': 1000,
  'learning_rate': 0.01,
  'l2_leaf_reg': 0.2},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
              

In [12]:
from sklearn.metrics import r2_score
pred = model.predict(X_test)

In [14]:
for i in range(len(pred)):
  if(pred[i]<0):
    pred[i]=0

In [15]:
pd.DataFrame({'revenue':pred}).to_csv("final_sub.csv")